<h1>SI 649 Midterm Project</h1>
<b>Haley Johnson</b></br>
<p>Code to create static visualization</p>

In [1]:
import pandas as pd
import numpy as np 
import os
import altair as alt
import tabula 
import html5lib

In [2]:
pd.options.mode.chained_assignment = None

<h2>Load in and preprocess Data</h2>

In [3]:
type_dfs = []
files = [f for f in os.listdir("../data") if f.endswith('html')]

for file in files:
    by_disability = pd.read_html(f"../data/{file}")[-2]
    type_dfs.append(by_disability)

disability_df = pd.concat(type_dfs).reset_index(drop = True)

In [5]:
disability_df['pct_uninsured'] = 1.0 - disability_df['Percent'].astype(float) / 100.0

disability_df = disability_df.rename(columns = {'Disability Type': 'type'})

<h2>Create Visualization</h2>

In [6]:
info_box_str =   r'''The Affordable Care Act, also known as ObamaCare, was passed by congress in 2010. Among other provisions, the \n 
    ACA made it illegal for insurance companies to deny coverage based on pre-existing conditions. Before this, individuals \n
    with disabilities or other chronic health conditions could be denied coverage. Likewise, insurance providers may not \n 
    charge higher premiums on account of disabilities under new ACA rules. The ACA has significantly expanded insurance \n
    options for Americans with disabilities, which will hopefully improve their access to low-cost high-quality care from\n
    providers that are informed about disabilities and sensitive to disabled American's unique care needs. 
    \n\nSee more at: https://tinyurl.com/ACADisability'''

In [8]:
# fix rules at certain years
aca_df = pd.DataFrame({'year': [2010]})
aca_effect_df = pd.DataFrame({'year': [2014]})

# color scheme for types plot 
type_colors = ['lightblue', 'lightblue', 'lightblue', 'lightblue']

title = alt.TitleParams('Percent of Disabled Americans Without Insurance', 
                        subtitle = 'The percent of disabled people without insurnace has decreased by nearly 50% since the passage of the Affordable Care Act',
                        anchor = 'start',
                        dx = 45, 
                        fontSize = 22, 
                        subtitleFontSize = 14)

all_lines = alt.Chart(disability_df, title = title).transform_filter(
    alt.datum.type == 'Any Disability'
).mark_line(
    strokeWidth = 5
).encode(
    x = alt.X('year:O', axis = alt.Axis(title = 'Year', labelFontSize = 12, titleFontSize = 15)), 
    y = alt.Y('pct_uninsured:Q', 
              axis = alt.Axis(format = '%', title = 'Percent Uninsured', labelFontSize = 12, titleFontSize = 15)), 
    color = alt.Color('type:N', scale = alt.Scale(range = ['#712ac7']), 
                      legend = alt.Legend(title = 'Type of Disability', titleFontSize = 15, 
                                          labelFontSize = 12, labelFontWeight = 'bold', 
                                          symbolStrokeWidth = 10, symbolSize = 200,
                                          orient = "none", legendX = 815, legendY = 0))
)

type_lines = alt.Chart(disability_df).transform_filter(
    (alt.datum.type != 'Any Disability') & (alt.datum.type != 'No Disability') & 
    (alt.datum.type != 'Self-Care') & (alt.datum.type != 'Independent Living')
).mark_line(
    strokeWidth = 1.5
).encode(
    x = alt.X('year:O', axis = alt.Axis(title = 'Year')), 
    y = alt.Y('pct_uninsured:Q', axis = alt.Axis(format = '%', title = 'Percent Uninsured')), 
    color = alt.Color('type:N', scale = alt.Scale(range = type_colors), legend = None), 
    
    shape = alt.Shape('type:N', scale = alt.Scale(range = ['cross', 'circle','triangle-right', 'diamond']), 
                       legend = alt.Legend(title = '', labelFontSize = 12, labelOffset = 11,
                                           symbolFillColor = 'lightblue', symbolSize = 150,
                                           symbolStrokeColor = 'lightblue', orient = "none", 
                                           legendX = 819, legendY = 44)
                     )
)

aca_passage = alt.Chart(aca_df).mark_rule( 
    color = "#4254f5",
    size = 2.5
).encode(
    x = alt.X('year:O')
)

aca_passage_text = alt.Chart(aca_df).mark_text(
    lineBreak = r'\n',
    text = r"2010: Passage of \nthe Affordable Care Act",
    align = 'left',
    fontSize = 13.5, 
    fontWeight = 'bold',
    dy = -40,
    dx = -225
)

aca_effect = alt.Chart(aca_effect_df).mark_rule(
    color = "#4254f5",
    size = 2.5
).encode(
    x = alt.X('year:O')
)

aca_effect_text = alt.Chart(aca_effect_df).mark_text(
    lineBreak = r'\n',
    text = r'2014: Most ACA \nProvisions Go Into Effect', 
    align = 'left',
    fontSize = 13.5, 
    fontWeight = 'bold',
    dy = 95,
    dx = 45
)

layers = alt.layer(
    type_lines, all_lines, aca_passage, aca_effect, aca_passage_text, aca_effect_text
).resolve_scale(
    color = 'independent'
).properties(
    width = 800, 
    height = 500
)

box = alt.Chart({'values':[{}]}
).mark_rect(
    stroke = 'lightblue',
    strokeWidth = 2.5,
    color = '#f2f3f5'
).encode(
    x = alt.value(0),
    x2 = alt.value(800),
    y = alt.value(0),
    y2 = alt.value(155))

text = alt.Chart({'values':[{}]}
).mark_text(
    align = "left", 
    baseline = "top",
    fontSize = 14.5, 
    lineBreak = r"\n"
).encode(
    x = alt.value(10),  
    y = alt.value(10), 
    text = alt.value(info_box_str)
)

text_box = alt.layer(box, text)

alt.vconcat(layers, text_box, padding = 0).configure_point(size = 90)

alt.VConcatChart(...)